# Introduction

This notebooks showcases a simpler parser for imonnit data, as obtained by the `Request raw data from IMONNIT API` Python Notebook.

# 1. Required Input Parameters

First we load the xml file provided by the previous Notebook:

In [2]:
from xml.dom.minidom import parse, parseString

dom = parse('imonnit_data.xml')
print(dom.toprettyxml(indent="\t"))

<?xml version="1.0" ?>
<SensorRestAPI xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
	
  
	<Method>SensorDataMessages</Method>
	
  
	<Result xsi:type="xsd:collection">
		<APIDataMessageList>
			
  
			<APIDataMessage Battery="100" Data="48" DataMessageGUID="22b9423d-0272-442d-8344-1e6e982bb05f" DataTypes="TemperatureData" DataValues="48" DisplayData="118.4° F" GatewayID="112709" MessageDate="10/23/2019 10:48:51 PM" MetNotificationRequirements="False" PlotLabels="Fahrenheit" PlotValue="118.4" PlotValues="118.4" SensorID="513030" SignalStrength="81" State="16" Voltage="3.06"/>
			
  
			<APIDataMessage Battery="100" Data="45.9" DataMessageGUID="c759ae7d-9831-4cf2-83f3-0150131246aa" DataTypes="TemperatureData" DataValues="45.9" DisplayData="114.6° F" GatewayID="112709" MessageDate="10/23/2019 8:49:32 PM" MetNotificationRequirements="False" PlotLabels="Fahrenheit" PlotValue="114.62" PlotValues="114.62" SensorID="513030" SignalStrength="8

We can see from the printed text above that the XML file containst the following schema:

```
<SensorRestAPI>
    <Method>
    <Result> 
        <APIDataMessageList>
            <APIDataMessage Battery Data DataMessageGUID DataTypes DataValues DisplayData GatewayID MessageDate MetNotificationRequirements PlotLabels PlotValue PlotValues SensorID SignalStrength State Voltage />
            <APIDataMessage> ...
            .
            .
            .
            <APIDataMessage>
        </APIDataMessageList>
    </Result>
</SensorRestAPI>
```

To us, the relevant data lies within the <APIDataMessage> nodes, each representing a row worth of data, and having as it's attributes the associated row elements. Sensor 513030 used in this example measures the water temperature. We extract all the information of each row with the following code block:

In [25]:
import pandas as pd 
import pendulum

def convert_F_to_C(temperature):
    return(float(temperature)*1.8 + 32)

rows = dom.getElementsByTagName('APIDataMessage')

rows_parsed = []

for row in rows:
    data = convert_F_to_C(row.getAttribute('Data'))
    signal = row.getAttribute('SignalStrength')
    datetime = pendulum.parse(row.getAttribute('MessageDate'))
    rows_parsed.append((datetime,data,signal))
    
df = pd.DataFrame.from_records(rows_parsed)
df.columns = ['datetime','reading','signal']
df['datetime'] = df['datetime'].dt.tz_convert('HST')
df['datetime_no_tz'] = df['datetime'].dt.tz_localize(None)
df

,datetime,reading,signal,datetime_no_tz
0,2019-10-23 12:48:51-10:00,118.40,81,2019-10-23 12:48:51
1,2019-10-23 10:49:32-10:00,114.62,80,2019-10-23 10:49:32
2,2019-10-23 08:50:33-10:00,114.80,81,2019-10-23 08:50:33
3,2019-10-23 06:51:19-10:00,115.16,78,2019-10-23 06:51:19
4,2019-10-23 04:51:29-10:00,115.70,77,2019-10-23 04:51:29
5,2019-10-23 02:51:42-10:00,116.42,77,2019-10-23 02:51:42
6,2019-10-23 00:51:37-10:00,116.96,77,2019-10-23 00:51:37
7,2019-10-22 22:51:33-10:00,117.50,77,2019-10-22 22:51:33
8,2019-10-22 20:51:35-10:00,118.22,77,2019-10-22 20:51:35
9,2019-10-22 18:51:26-10:00,118.76,76,2019-10-22 18:51:26
